# Importando Dependências

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from keras import utils
from keras import callbacks
from keras import optimizers

sequential = models.Sequential
dense = layers.Dense
to_categorical = utils.to_categorical



2024-06-16 19:47:58.996520: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-16 19:47:59.004043: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-16 19:47:59.108246: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-16 19:48:00.263260: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
df = pd.read_csv('training' + str(4) + ".csv")

df['track_genre'].unique()

array(['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient',
       'anime', 'black-metal', 'bluegrass', 'blues', 'brazil',
       'breakbeat', 'british', 'cantopop', 'chicago-house', 'children',
       'chill', 'classical', 'club', 'comedy', 'country', 'dance',
       'dancehall', 'death-metal', 'deep-house', 'detroit-techno',
       'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm',
       'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk',
       'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove',
       'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle',
       'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'idm', 'indian',
       'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol',
       'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino',
       'malay', 'mandopop', 'metal', 'metalcore', 'minimal-techno', 'mpb',
       'new-age', 'opera', 'pagode', 'party', 'piano', 'pop', 'pop-film',
       'pow

In [10]:
114 % 4

2

# Treinando Rede Neural

In [11]:
def neuralTraining(index, optimize):
    # Load the dataset
    df = pd.read_csv('training' + str(index) + ".csv")

    # Select features and labels
    features = ['popularity', 'duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
    # features = ['duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
    X = df[features].values
    y = df['track_genre']

    print(y)

    training_labels = y

    # Encode the labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    X_train_copy = X_train

    # Feature Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    len(y_train)


    # Reduzir LR quando acurácia não melhora
    reduce_lr_callback = callbacks.ReduceLROnPlateau(
        "accuracy",
        factor = 0.5,
        patience=3,
        verbose=1,
        min_delta=0.002,
        min_lr=0.002,
    )
    early_stop_callback = callbacks.EarlyStopping(
        "accuracy",
        patience=7,
        verbose=1,
    )
    opt = optimizers.Adam(
        learning_rate=0.003
    )

    # Convert labels to categorical
    y_train = to_categorical(y_train, num_classes=114)
    y_test = to_categorical(y_test, num_classes=114)

    print

    # Define the model
    model = sequential()
    model.add(dense(128, input_dim=X_train.shape[1], activation='relu'))
    model.add(dense(64, activation='relu'))
    model.add(dense(32, activation='relu'))
    model.add(dense(114, activation='softmax'))

    # Compile the model
    if(optimize == True):
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    else:
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    if(optimize == True):
        model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[reduce_lr_callback, early_stop_callback])
    else:
        model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'Test Accuracy: {accuracy*100:.2f}%')
    return model, df[features].values, training_labels

model, X_train, training_labels = neuralTraining(4, True)

0           acoustic
1           acoustic
2           acoustic
3           acoustic
4           acoustic
            ...     
45595    world-music
45596    world-music
45597    world-music
45598    world-music
45599    world-music
Name: track_genre, Length: 45600, dtype: object
Epoch 1/50


/home/guilhermepacianotto/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1140/1140 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.1165 - loss: 3.8478 - val_accuracy: 0.2404 - val_loss: 3.0185 - learning_rate: 0.0030
Epoch 2/50
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2612 - loss: 2.9115 - val_accuracy: 0.2985 - val_loss: 2.7397 - learning_rate: 0.0030
Epoch 3/50
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3045 - loss: 2.6469 - val_accuracy: 0.3177 - val_loss: 2.6001 - learning_rate: 0.0030
Epoch 4/50
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3325 - loss: 2.5086 - val_accuracy: 0.3402 - val_loss: 2.5086 - learning_rate: 0.0030
Epoch 5/50
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3481 - loss: 2.4432 - val_accuracy: 0.3395 - val_loss: 2.5108 - learning_rate: 0.0030
Epoch 6/50
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3558 - loss: 2.3948 - val_accuracy: 0.3542 - val_loss: 2.4553 - learning_rate: 0.0030
Epoch 7/50
1140/1140 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3658 - loss: 2.3463

In [12]:
model.save("song_classifier.keras")

def ndarray_to_csv(array, csv_file_path):
    """
    Transform a NumPy ndarray to a Pandas DataFrame and save it to a CSV file.

    Parameters:
    array (np.ndarray): The input ndarray.
    csv_file_path (str): The file path to save the CSV.
    """
    # Convert ndarray to DataFrame
    df = pd.DataFrame(array)
    
    # Save DataFrame to CSV
    df.to_csv(csv_file_path, index=False)
    print(f"Array saved to {csv_file_path}")

ndarray_to_csv(X_train, "X_train.csv")
ndarray_to_csv(training_labels, "training_labels.csv")

Array saved to X_train.csv
Array saved to training_labels.csv


# Medindo Precisão por Gênero

# Treinando uma decision Tree

In [19]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics

def decisionTreeTraining(index):
    df = pd.read_csv('training' + str(index) + ".csv")

    # Select features and labels
    features = ['popularity', 'duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
    # features = ['duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
    X = df[features].values
    y = df['track_genre']

    print(y)

    # Encode the labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Feature Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    # len(y_train)

    model_tree = DecisionTreeClassifier(max_depth=12, random_state=2)
    model_tree.fit(X_train, y_train)
    prediction = model_tree.predict(X_test)
    print('The accuracy of the Decision Tree is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test)))

decisionTreeTraining(4)

0           acoustic
1           acoustic
2           acoustic
3           acoustic
4           acoustic
            ...     
45595    world-music
45596    world-music
45597    world-music
45598    world-music
45599    world-music
Name: track_genre, Length: 45600, dtype: object
The accuracy of the Decision Tree is 0.312


# Treinando Com KNN

In [20]:
from sklearn.neighbors import KNeighborsClassifier

def knnTraining(index):
    df = pd.read_csv('training' + str(index) + ".csv")

    # Select features and labels
    features = ['popularity', 'duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
    # features = ['duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
    X = df[features].values
    y = df['track_genre']

    print(y)

    # Encode the labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Feature Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    mod_5nn=KNeighborsClassifier(n_neighbors=10, algorithm="ball_tree")
    mod_5nn.fit(X_train,y_train)
    prediction=mod_5nn.predict(X_test)
    print('The accuracy of the 5NN Classifier is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test)))

knnTraining(4)

0           acoustic
1           acoustic
2           acoustic
3           acoustic
4           acoustic
            ...     
45595    world-music
45596    world-music
45597    world-music
45598    world-music
45599    world-music
Name: track_genre, Length: 45600, dtype: object
The accuracy of the 5NN Classifier is 0.203


# Guassian Naive Bayes Classifier

In [21]:
from sklearn.naive_bayes import GaussianNB

def gaussianTraining(index):
    df = pd.read_csv('training' + str(index) + ".csv")

    # Select features and labels
    features = ['popularity', 'duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
    # features = ['duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
    X = df[features].values
    y = df['track_genre']

    print(y)

    # Encode the labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Feature Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    mod_gnb_all = GaussianNB()
    y_pred = mod_gnb_all.fit(X_train, y_train).predict(X_test)
    print('The accuracy of the Guassian Naive Bayes Classifier on test data is',"{:.3f}".format(metrics.accuracy_score(y_pred,y_test)))

gaussianTraining(4)

0           acoustic
1           acoustic
2           acoustic
3           acoustic
4           acoustic
            ...     
45595    world-music
45596    world-music
45597    world-music
45598    world-music
45599    world-music
Name: track_genre, Length: 45600, dtype: object
The accuracy of the Guassian Naive Bayes Classifier on test data is 0.198


# LDA

In [22]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

def ldaTraining(index):
    df = pd.read_csv('training' + str(index) + ".csv")

    # Select features and labels
    features = ['popularity', 'duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
    # features = ['duration_ms', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
    X = df[features].values
    y = df['track_genre']

    print(y)

    # Encode the labels
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Feature Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    mod_lda_all = LinearDiscriminantAnalysis(solver="svd", shrinkage=None)
    y_pred = mod_lda_all.fit(X_train, y_train).predict(X_test)
    print('The accuracy of the LDA Classifier on test data is',"{:.3f}".format(metrics.accuracy_score(y_pred,y_test)))

ldaTraining(4)

0           acoustic
1           acoustic
2           acoustic
3           acoustic
4           acoustic
            ...     
45595    world-music
45596    world-music
45597    world-music
45598    world-music
45599    world-music
Name: track_genre, Length: 45600, dtype: object
The accuracy of the LDA Classifier on test data is 0.184


Pesquisar: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier